In [30]:
import pickle as pickle
import os
import pandas as pd
import torch
from utils.utils import entity_marker, typed_entity_marker, typed_entity_marker_punc, TYPE_MARKERS

In [2]:
class RE_Dataset(torch.utils.data.Dataset):
    """ Dataset 구성을 위한 class."""
    def __init__(self, pair_dataset, labels):
        self.pair_dataset = pair_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [26]:
def preprocessing_dataset(dataset):
    """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    # subject_entity = []
    # object_entity = []
    # for i,j in zip(dataset['subject_word'], dataset['object_word']):
    #     i = i[1:-1].split(',')[0].split(':')[1]
    #     j = j[1:-1].split(',')[0].split(':')[1]

    #     subject_entity.append(i)
    #     object_entity.append(j)
    typed_sentence = []
    for i, data in dataset.iterrows():
        typed_sentence.append(typed_entity_marker(data))
        # print(data['sentence'])
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':typed_sentence,'subject_entity':dataset['subject_word'],'object_entity':dataset['object_word'],'label':dataset['label'],})
    return out_dataset

In [27]:
def load_data(dataset_dir):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)
    
    return dataset

In [42]:
def tokenized_dataset(dataset, tokenizer):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
        
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens" : list(TYPE_MARKERS.values())})
    print("We have added", num_added_toks, "tokens")

    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
        )
    return tokenized_sentences

In [43]:
from transformers import AutoTokenizer
MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = load_data('../../dataAugmentation/entity_split.csv')
# print(train_dataset)
tokenized_train = tokenized_dataset(train_dataset, tokenizer)


          id                                           sentence  \
0          0  〈Something〉는 <O:PER> 조지 해리슨 </O:PER>이 쓰고 <S:OR...   
1          1  호남이 기반인 바른미래당·<O:ORG> 대안신당 </O:ORG>·<S:ORG> 민주...   
2          2  K리그2에서 성적 1위를 달리고 있는 <S:ORG> 광주FC </S:ORG>는 지난...   
3          3  균일가 생활용품점 (주)<S:ORG> 아성다이소 </S:ORG>(대표 <O:PER>...   
4          4  <O:DAT> 1967 </O:DAT>년 프로 야구 드래프트 1순위로 <S:ORG>...   
...      ...                                                ...   
32465  32465  한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...   
32466  32466  법포는 다시 <S:PER> 최시형 </S:PER>, 서병학, <O:PER> 손병희 ...   
32467  32467  <S:ORG> 완도군 </S:ORG>(군수 <O:PER> 신우철 </O:PER>)이...   
32468  32468  중앙일보, <S:ORG> JTBC </S:ORG> 회장을 지낸 이후 <O:ORG> ...   
32469  32469  <S:ORG> 화순군 </S:ORG>(군수 <O:PER> 구충곤 </O:PER>)은...   

      subject_entity object_entity                      label  
0                비틀즈        조지 해리슨                no_relation  
1              민주평화당          대안신당                no_relation  
2  

tensor([    0, 29830,     2,  8373, 14113,  2234,     2,   168, 30985, 14451,
         7088,  4586,   169,   793, 32000,  8373, 14113,  2234, 32011,  1504,
         1363,  2088, 32012, 29830, 32013,   543, 14879,  2440,  6711,   170,
        21406, 26713,  2076, 25145,  5749,   171,  1421,   818,  2073,  4388,
         2062,    18,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [ ]:
tokenized_train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [45]:
tokenizer.decode(tokenized_train['input_ids'][0])

'[CLS] 비틀즈 [SEP] 조지 해리슨 [SEP] 〈 Something 〉 는 <O:PER> 조지 해리슨 </O:PER> 이 쓰고 <S:ORG> 비틀즈 </S:ORG> 가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [46]:
leng = []
for s in train_dataset['sentence']:
    leng.append(len(tokenizer.tokenize(s)))
leng

[35,
 38,
 51,
 46,
 30,
 56,
 55,
 59,
 85,
 67,
 26,
 51,
 122,
 58,
 74,
 69,
 96,
 38,
 48,
 51,
 30,
 62,
 58,
 60,
 37,
 82,
 52,
 46,
 29,
 42,
 51,
 65,
 142,
 31,
 59,
 60,
 37,
 70,
 57,
 39,
 28,
 44,
 77,
 53,
 108,
 45,
 52,
 45,
 140,
 37,
 46,
 78,
 52,
 46,
 30,
 57,
 43,
 103,
 33,
 81,
 42,
 80,
 26,
 87,
 51,
 59,
 85,
 106,
 40,
 51,
 31,
 43,
 47,
 70,
 90,
 38,
 62,
 50,
 78,
 71,
 35,
 45,
 51,
 64,
 63,
 77,
 23,
 79,
 80,
 38,
 43,
 49,
 68,
 57,
 57,
 44,
 31,
 48,
 32,
 77,
 53,
 50,
 35,
 113,
 66,
 16,
 39,
 85,
 52,
 47,
 74,
 69,
 73,
 50,
 54,
 110,
 32,
 45,
 88,
 39,
 36,
 31,
 76,
 51,
 114,
 23,
 55,
 49,
 87,
 59,
 106,
 49,
 40,
 34,
 43,
 59,
 34,
 26,
 59,
 87,
 27,
 106,
 46,
 72,
 28,
 51,
 61,
 48,
 18,
 53,
 38,
 43,
 36,
 41,
 55,
 70,
 39,
 62,
 74,
 35,
 19,
 59,
 30,
 46,
 44,
 40,
 51,
 70,
 101,
 35,
 131,
 30,
 85,
 49,
 31,
 40,
 62,
 62,
 38,
 67,
 48,
 59,
 33,
 53,
 49,
 33,
 42,
 43,
 63,
 28,
 61,
 34,
 56,
 54,
 85,
 85,
 36,
 3

In [47]:
max(leng)

243

In [63]:
tokens = tokenizer.tokenize(train_dataset['sentence'][0])
tmp = [0 for _ in range(256)]
tmpidx = [[],[]]
print(tokens)
for i in range(len(tokens)):
    if '<S:' in tokens[i] or '<O:' in tokens[i]:
        print(tokens[i])
        tmpidx[0].append(i)
    elif '</S:' in tokens[i] or '</O:' in tokens[i]:
        print(tokens[i])
        tmpidx[1].append(i)
        
def update_ranges_to_1(lst, start_tokens, end_tokens):
    result = [0] * len(lst)
    stack = []

    for i, token in enumerate(lst):
        if token in start_tokens:
            stack.append(i)
        elif token in end_tokens and stack:
            start_pos = stack.pop()
            end_pos = i
            result[start_pos:end_pos + 1] = [1] * (end_pos - start_pos + 1)

    return result

update_ranges_to_1(tmp, tmpidx[0], tmpidx[1])


['〈', 'So', '##me', '##th', '##ing', '〉', '는', '<O:PER>', '조지', '해리', '##슨', '</O:PER>', '이', '쓰', '##고', '<S:ORG>', '비틀즈', '</S:ORG>', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']
<O:PER>
</O:PER>
<S:ORG>
</S:ORG>


TypeError: update_ranges_to_1() missing 1 required positional argument: 'end_tokens'